##### movies-clean-transform notebook
***

<h1>Clean and Transform</h1>

### The purpose of this notebook is to:
    1. View data types
    2. Convert data types where needed
    3. Deal with missing values
    4. Inspect table relationships
    5. Potentially:
        a. Create derived values where advantageous
        b. Make initial joins
        c. Drop irrelevant tables

***

#### import required libraries

In [1]:
import os # for setting the current directory

import numpy as np
import pandas as pd

import sqlite3

import pandasql

In [2]:
# set the current working directory
os.chdir("c:/users/jd/flatiron/project01/dsc-mod-1-project-v2-1-online-ds-ft-120919/")

# print the current working directory
print(os.getcwd())

c:\users\jd\flatiron\project01\dsc-mod-1-project-v2-1-online-ds-ft-120919


#### connect to and preview sqlite database

In [3]:
# connect to sql movies_db data source and instantiate a cursor
conn = sqlite3.connect("movies_db.sqlite")
cur = conn.cursor()

#### import helper functions

In [4]:
from importlib import reload

import helper_functions as hf

In [5]:
# `helper_functions` includes `get_table_list(conn)`, 
# `load_table(conn, table_name)` and `convert_dollars_to_int(df, col)
# note: `get_table_list(conn)` result the same as the above `table_list`
reload(hf)

<module 'helper_functions' from 'c:\\users\\jd\\flatiron\\project01\\dsc-mod-1-project-v2-1-online-ds-ft-120919\\helper_functions.py'>

In [6]:
# function to preview all tables from sqlite_master
# or a sub - set entered as a list
def preview_tables(conn, tables='all'):
    all_tables = hf.get_table_list(conn)
    if tables=='all':
        final_table_names = all_tables
        
    elif type(tables) == list:
        final_table_names = [t for t in all_tables if t in tables]
    
    for table_name in final_table_names:
        print(f"Showing Table: {table_name}")
        query = f"select * from {table_name};"
        df = pd.read_sql(query, conn)
        display(df.head(2))
        display(df.info())
        print("-"*100)
    print("finished")
    return None

<h3 align='center'><font color='coral'>INSPECT TABLES AND DATA TYPES</font></h3>

In [7]:
preview_tables(conn)

Showing Table: bom_movie_gross


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


None

----------------------------------------------------------------------------------------------------
Showing Table: imdb_name_basics


...

----------------------------------------------------------------------------------------------------
Showing Table: tn_movie_budgets


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


None

----------------------------------------------------------------------------------------------------
finished


#### Preview suggests columns of interest (based on our questions) regarding box office performance for titles, studios, and genres
   
    >  `bom_movie_gross`: ['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']
    >  `tn_movie_budgets`: ['release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross']
    >  `imdb_title_basics`: ['primary_title', 'start_year', 'genres']
    >  `rt_movie_info`: ['genre', 'theater_date', 'currency', 'box_office']    

#### Cleaning Notes:
* bom_movie_gross | 'domestic_gross' is a REAL number, while 'foreign_gross' is TEXT; table has no foreign keys (unless title names match)
* tn_movie_budgets | 'production_budget', 'domestic_gross', and 'worldwide_gross' are each TEXT
* imdb_title_basics | 'genres' values are "," separated
* rt_movie_info | 'box_office' is TEXT; 'genre' values are "|" separated
* all "*date*" values are text, while "*year*" values are of type INTEGER

#### We will review and clean these tables. in turn, below.

<h2 align='center'><font color='chocolate'>SEQUENTIALLY REVIEW, CLEAN, AND TRANSFORM</font></h2>

### bom_movie_gross

In [47]:
preview_tables(conn, ['bom_movie_gross'])

Showing Table: bom_movie_gross


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


None

----------------------------------------------------------------------------------------------------
finished


#### Change `foreign_gross` data type to match `domestic_gross`

### Connect `bom_movie_gross`

In [9]:
# connect to the table and view in pandas for cleaning
cur.execute('''SELECT *
                    FROM bom_movie_gross
                    ;''')

clean_bom_df = pd.DataFrame(cur.fetchall())
clean_bom_df.columns = [x[0] for x in cur.description]
display(clean_bom_df.dtypes)
clean_bom_df.head(3)

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010


### Convert  `clean_bom_df`

In [10]:
# convert `foreign_gross` to remove commas and to match `domestic_gross` data type
clean_bom_df['foreign_gross'] = clean_bom_df['foreign_gross'].str.replace(',', '').astype(float)

# add a `Total_gross` column for EDA
clean_bom_df['Total_gross'] = clean_bom_df['domestic_gross'] + clean_bom_df['foreign_gross']

# and view a sample
clean_bom_df.loc[clean_bom_df.title == 'Alice in Wonderland (2010)', :]

,title,studio,domestic_gross,foreign_gross,year,Total_gross
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09


In [11]:
clean_bom_df.info()
# data type converted and column added
# let's check for null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 6 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null float64
year              3387 non-null int64
Total_gross       2009 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 158.9+ KB


In [12]:
# we have null values--first drop where all values are null...
clean_bom_df.dropna(how = 'all')
clean_bom_df.info()

# no change--no rows with null values across the board

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 6 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null float64
year              3387 non-null int64
Total_gross       2009 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 158.9+ KB


In [13]:
# and create a function to count where we still have them
def null_count(s):
    null_columns=s.columns[s.isnull().any()]
    return s[null_columns].isnull().sum()

In [17]:
display(null_count(clean_bom_df))

# also preview rows where domestic_gross is null and where `studio` is null
display(clean_bom_df[pd.notnull(clean_bom_df['domestic_gross']) != True].sort_values(by='year'))
display(clean_bom_df[pd.notnull(clean_bom_df['studio']) != True].sort_values(by='year'))

# we can drop rows for movies without a studio
# and change 'gross' series nulls to 0

studio               5
domestic_gross      28
foreign_gross     1350
Total_gross       1378
dtype: int64

,title,studio,domestic_gross,foreign_gross,year,Total_gross
230,It's a Wonderful Afterlife,UTV,NaN,1300000.0,2010,NaN
298,Celine: Through the Eyes of the World,Sony,NaN,119000.0,2010,NaN
302,White Lion,Scre.,NaN,99600.0,2010,NaN
306,Badmaash Company,Yash,NaN,64400.0,2010,NaN
327,Aashayein (Wishes),Relbig.,NaN,3800.0,2010,NaN
537,Force,FoxS,NaN,4800000.0,2011,NaN
713,Empire of Silver,NeoC,NaN,19000.0,2011,NaN
1017,Dark Tide,WHE,NaN,432000.0,2012,NaN
966,The Cup (2012),Myr.,NaN,1800000.0,2012,NaN
936,"Lula, Son of Brazil",NYer,NaN,3800000.0,2012,NaN


,title,studio,domestic_gross,foreign_gross,year,Total_gross
210,Outside the Law (Hors-la-loi),None,96900.0,3300000.0,2010,3396900.0
555,Fireflies in the Garden,None,70600.0,3300000.0,2011,3370600.0
933,Keith Lemon: The Film,None,NaN,4000000.0,2012,NaN
1862,Plot for Peace,None,7100.0,NaN,2014,NaN
2825,Secret Superstar,None,NaN,122000000.0,2017,NaN


In [19]:
# drop null `studio` rows (if we can drop 'None' values)
clean_bom_df.dropna(subset=['studio'], inplace=True)
display(clean_bom_df[pd.notnull(clean_bom_df['studio']) != True])

,title,studio,domestic_gross,foreign_gross,year,Total_gross


In [20]:
# ...and for `domestic_gross` nulls
clean_bom_df.domestic_gross.fillna(0, inplace=True)

# check with the same line we used to view nulls above
display(clean_bom_df[pd.notnull(clean_bom_df['domestic_gross']) != True].sort_values(by='year'))

,title,studio,domestic_gross,foreign_gross,year,Total_gross


In [21]:
# sanity check
null_count(clean_bom_df)

foreign_gross    1349
Total_gross      1375
dtype: int64

In [ ]:
#  create a function to replace series null values with 0
# def fill_na_zeroes(series):
#     series.fillna(0)
#     print("-+-+-+-+-+-Null values replaced with '0'.-+-+-+-+-+-")

In [ ]:
# fill_na_zeroes(clean_bom_df.domestic_gross)
# fill_na_zeroes(clean_bom_df.foreign_gross)

# display(clean_bom_df.domestic_gross.isna().value_counts(), clean_bom_df.foreign_gross.isna().value_counts())

In [23]:
# It looks like we need to re-calculate the `Total_gross` column
clean_bom_df[clean_bom_df['title'] == "Empire of Silver"]

,title,studio,domestic_gross,foreign_gross,year,Total_gross
713,Empire of Silver,NeoC,0.0,19000.0,2011,NaN


In [24]:
# update `Total_gross` column for EDA
clean_bom_df['Total_gross'] = clean_bom_df['domestic_gross'] + clean_bom_df['foreign_gross']

# and view a sample
clean_bom_df[clean_bom_df['title'] == "Empire of Silver"]

,title,studio,domestic_gross,foreign_gross,year,Total_gross
713,Empire of Silver,NeoC,0.0,19000.0,2011,19000.0


### Add `clean_bom_df` to sqlite

In [25]:
# add the new df as a table to the sqlite database
clean_bom_df.to_sql('clean_bom_tbl', conn, if_exists='replace')

# view names of all tables in the sql database to verify operation
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('bom_movie_gross',),
 ('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',),
 ('rt_movie_info',),
 ('rt_reviews',),
 ('tmdb_movies',),
 ('tn_movie_budgets',),
 ('clean_bom_tbl',)]

### tn_movie_budgets

In [26]:
preview_tables(conn, ['tn_movie_budgets'])

Showing Table: tn_movie_budgets


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


None

----------------------------------------------------------------------------------------------------
finished


### Connect `tn_movie_budgets`

In [27]:
# connect to the table and view in pandas for cleaning
cur.execute('''SELECT *
                    FROM tn_movie_budgets
                    ;''')

clean_tn_budgets_df = pd.DataFrame(cur.fetchall())
clean_tn_budgets_df.columns = [x[0] for x in cur.description]
display(clean_tn_budgets_df.dtypes)
clean_tn_budgets_df.head(3)

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"


### Convert

In [28]:
# convert date
clean_tn_budgets_df.release_date = pd.to_datetime(clean_tn_budgets_df.release_date)
clean_tn_budgets_df['Year'] = clean_tn_budgets_df.release_date.map(lambda x: x.strftime('%Y'))
clean_tn_budgets_df.head(1)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,Year
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009


#### Our preview indicated no null values, but we do have some cleaning to do.
* We need large integer data types for the budget and gross fields.
* Symbols and separators must be removed.  Method chaining helps to keep the operation efficient.

In [32]:
# `convert_dollars_to_int(df, col)` accepts a dataframe and a column
# encountered errors
# we can quickly pass each of the necessary args via a loop

for col in clean_tn_budgets_df.columns[3:6]:
    clean_tn_budgets_df[col] = clean_tn_budgets_df[col].str.replace("$", "").str.replace(",", "").astype('float64')

clean_tn_budgets_df.dtypes

id                            int64
release_date         datetime64[ns]
movie                        object
production_budget           float64
domestic_gross              float64
worldwide_gross             float64
Year                         object
dtype: object

In [33]:
# view the top - ten in order of `worldwide_gross`
display(clean_tn_budgets_df.head(10).sort_values(by=['worldwide_gross'], ascending=False))

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,Year
0,1,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2009
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000.0,936662225.0,2.053311e+09,2015
6,7,2018-04-27,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,2018
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011
7,8,2007-05-24,Pirates of the Caribbean: At Worldâs End,300000000.0,309420425.0,9.634204e+08,2007
9,10,2015-11-06,Spectre,300000000.0,200074175.0,8.796209e+08,2015
8,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017
2,3,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019


In [36]:
# add a column to display return on investment (ROI)
clean_tn_budgets_df['ROI'] = round((
    (clean_tn_budgets_df['worldwide_gross'] - clean_tn_budgets_df["production_budget"]
    ) / clean_tn_budgets_df['worldwide_gross']) * 100, 2).astype('float64')

display(clean_tn_budgets_df.dtypes)
clean_tn_budgets_df.head(3)

id                            int64
release_date         datetime64[ns]
movie                        object
production_budget           float64
domestic_gross              float64
worldwide_gross             float64
Year                         object
ROI                         float64
dtype: object

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,Year,ROI
0,1,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2009,84.69
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,60.73
2,3,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019,-133.70


### Add `clean_tn_budgets_df` to sqlite

In [37]:
# add the new df as a table to the sqlite database
clean_tn_budgets_df.to_sql('clean_tn_tbl', conn, if_exists='replace')

# view names of all tables in the sql database to verify operation
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('bom_movie_gross',),
 ('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',),
 ('rt_movie_info',),
 ('rt_reviews',),
 ('tmdb_movies',),
 ('tn_movie_budgets',),
 ('clean_bom_tbl',),
 ('clean_tn_tbl',)]

### imdb_title_basics

In [38]:
preview_tables(conn, ['imdb_title_basics'])

Showing Table: imdb_title_basics


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
tconst             146144 non-null object
primary_title      146144 non-null object
original_title     146123 non-null object
start_year         146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


None

----------------------------------------------------------------------------------------------------
finished


### Connect `imdb_title_basics`

In [48]:
# connect to the table and view in pandas for cleaning
cur.execute('''SELECT *
                    FROM imdb_title_basics
                    ;''')

clean_imdb_title_df = pd.DataFrame(cur.fetchall())
clean_imdb_title_df.columns = [x[0] for x in cur.description]
display(clean_imdb_title_df.dtypes)
clean_imdb_title_df.head(3)

tconst              object
primary_title       object
original_title      object
start_year           int64
runtime_minutes    float64
genres              object
dtype: object

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama


### Extract and expand `clean_imdb_title_df` genres

In [57]:

'''
Declare and initialize an empty list, SEPS,
split the items in each row of `genres`
to a one-word-list, and extend each onto
the end of SEPS
'''
######
SEPS = []
for thing in clean_imdb_title_df.genres:
    SEPA = str(thing).split(",")
    SEPS.extend(SEPA)
display(SEPS[:5]) # first 5 items in result
display(len(SEPS))
display(len(set(SEPS))) # number or unique items
#####################################

['Action', 'Crime', 'Drama', 'Biography', 'Drama']

234958

28

In [59]:
# down to 28 unique words
# let's initialize a variable, and assign them to a list, and view them
title_genres = list(set(SEPS))
title_genres

['Reality-TV',
 'Music',
 'Western',
 'Short',
 'Sport',
 'Game-Show',
 'Biography',
 'Musical',
 'History',
 'Adventure',
 'Adult',
 'None',
 'War',
 'Drama',
 'Talk-Show',
 'Horror',
 'News',
 'Family',
 'Mystery',
 'Animation',
 'Comedy',
 'Action',
 'Romance',
 'Sci-Fi',
 'Crime',
 'Thriller',
 'Fantasy',
 'Documentary']

In [ ]:
clean_imdb_title_df.head()

In [ ]:
# collect the genres to a list
# all_genres = []
# for genres in clean_imdb_title_df.genres:
#     for genre in genres: # each item from each genres sublist
#         if genre not in all_genres:
#             all_genres.append(genre) # add to the list

# display(len(all_genres), all_genres)

In [ ]:
clean_imdb_title_df.head()

In [ ]:
# split_2 = []

# for index, row in clean_imdb_title_df.iterrows():
#     if index % 1000 == 0:
#         print(index)
#     split_words = str(row['genres']).split()
#     for word in split_words:
#         if word not in split_2:
#             split_2.append((index, word))
#             clean_imdb_title_df.loc[index, word] = 1

In [ ]:
# display(len(split_2))
# split_2[:35]

In [ ]:
# add the dataframe's genres to a list
# initialize and empty list
outer_list = []

# convert the `genres` series arrays to nested lists
# for row in clean_imdb_title_df.genres:
#     outer_list.append(row)

# print("nested:\n", outer_list[:5])
# print("\nsublist item:", outer_list[1][1])

# title_genres = []

# print(type(outer_list[1]))

# for sublist in outer_list:
#     for word in sublist:
#         if word not in title_genres:
#             title_genres.append(word)

# title_genres
            
# flatten the list
# flat_genres = []

# finally, convert the flat list to a unique list
# genre_list = list(dict.fromkeys(flat_genres))
# print("\n", len(genre_list), "items in new `genre_list`:\n", genre_list)

# for sublist in outer_list:
#     for item in sublist: # inner list items
#         flat_genres.append(str(item).split(","))


In [ ]:
# create column for each genre
# to count 1 if present for a title


## ////////////////////These genres are a challenge\\\\\\\\\\\\\\\\\\\\

### Add `imdb_title_basics` to sqlite

In [62]:
# add the new df as a table to the sqlite database
clean_imdb_title_df.to_sql('clean_title_tbl', conn, if_exists='replace')

# view names of all tables in the sql database to verify operation
# conn.execute("select name from sqlite_master where type='table';").fetchall()

### `rt_movie_info`

In [39]:
preview_tables(conn, ['rt_movie_info'])

Showing Table: rt_movie_info


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",None,None,104 minutes,None
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
id              1560 non-null int64
synopsis        1498 non-null object
rating          1557 non-null object
genre           1552 non-null object
director        1361 non-null object
writer          1111 non-null object
theater_date    1201 non-null object
dvd_date        1201 non-null object
currency        340 non-null object
box_office      340 non-null object
runtime         1530 non-null object
studio          494 non-null object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


None

----------------------------------------------------------------------------------------------------
finished


### Connect `rt_movie_info`

In [63]:
# connect to the table and view in pandas for cleaning
cur.execute('''SELECT *
                    FROM rt_movie_info
                    ;''')

clean_rt_info_df = pd.DataFrame(cur.fetchall())
clean_rt_info_df.columns = [x[0] for x in cur.description]
display(clean_rt_info_df.dtypes)
clean_rt_info_df.head(3)

id               int64
synopsis        object
rating          object
genre           object
director        object
writer          object
theater_date    object
dvd_date        object
currency        object
box_office      object
runtime         object
studio          object
dtype: object

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",None,None,104 minutes,None
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",None,None,116 minutes,None


In [64]:
# let's see what shape the dataframe is in
print("rows and columns:", clean_rt_info_df.shape)

# how many values are missing
print("number of 'box_office' missing values:", clean_rt_info_df['box_office'].isna().sum())

# we may as well take a deeper look while we're at it
display(clean_rt_info_df.head(5).sort_values(by=['box_office'], ascending=False))

rows and columns: (1560, 12)
number of 'box_office' missing values: 1220


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",None,None,104 minutes,None
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",None,None,116 minutes,None
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",None,None,128 minutes,None
4,7,None,NR,Drama|Romance,Rodney Bennett,Giles Cooper,None,None,None,None,200 minutes,None


#### Whoa! That's a <i>rotten</i> number of missing values. And where are the movie titles?
#### This dataframe may only be of use if we break out those genre categories. 

### Convert `clean_rt_info_df`

In [43]:
# Create a list of unique genres for future use
genre_list = []

for cols in df1.columns:
    for v in df1[cols]: # check values in all columns
        if v != None: # disregard None values
            if v not in genre_list:
                genre_list.append(v) # add to the list

genre_list

['Action and Adventure',
 'Drama',
 'Comedy',
 'Documentary',
 'Classics',
 'Art House and International',
 'Musical and Performing Arts',
 'Horror',
 'Mystery and Suspense',
 'Animation',
 'Special Interest',
 'Kids and Family',
 'Western',
 'Science Fiction and Fantasy',
 'Romance',
 'Sports and Fitness',
 'Television',
 'Cult Movies',
 'Anime and Manga',
 'Faith and Spirituality',
 'Gay and Lesbian']

In [42]:
# split `rt_clean_df.genre` values to a new dataframe
df1 = clean_rt_info_df.genre.str.split("|", expand=True)
df1.columns = "g1 g2 g3 g4 g5 g6 g7".split()
df1

,g1,g2,g3,g4,g5,g6,g7
0,Action and Adventure,Classics,Drama,None,None,None,None
1,Drama,Science Fiction and Fantasy,None,None,None,None,None
2,Drama,Musical and Performing Arts,None,None,None,None,None
3,Drama,Mystery and Suspense,None,None,None,None,None
4,Drama,Romance,None,None,None,None,None
...,...,...,...,...,...,...,...
1555,Action and Adventure,Horror,Mystery and Suspense,None,None,None,None
1556,Comedy,Science Fiction and Fantasy,None,None,None,None,None
1557,Classics,Comedy,Drama,Musical and Performing Arts,None,None,None
1558,Comedy,Drama,Kids and Family,Sports and Fitness,None,None,None


In [ ]:
# split_3 = []

# for index, row in clean_rt_info_df.iterrows():
#     if index % 1000 == 0:
#         print(index)
#     split_words = str(row['genres']).split()
#     for word in split_words:
#         if word not in split_3:
#             split_3.append((index, word))
#             clean_rt_info_df.loc[index, word] = 1

#### Merge the df1 genre columns to  `clean_rt_info_df` as a new data frame.

In [ ]:
# join the two datframes to add genre columns to each row
# these dataframes share the same indices
# rt_genres = pd.merge(rt_clean_df, df1, left_index=True, right_index=True)
# display(rt_genres.head(1))

## what we actually want is a count of each genere for each movie

### Add `clean_rt_info_df to sqlite

In [46]:
# add the new df as a table to the sqlite database as-is
# we can return for neccessary adjustment
clean_rt_info_df.to_sql('clean_rt_tbl', conn, if_exists='replace')

# view names of all tables in the sql database to verify operation
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('bom_movie_gross',),
 ('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',),
 ('rt_movie_info',),
 ('rt_reviews',),
 ('tmdb_movies',),
 ('tn_movie_budgets',),
 ('clean_tn_tbl',),
 ('clean_bom_tbl',),
 ('clean_rt_tbl',)]

### `tmdb_movies`

In [45]:
preview_tables(conn, ['tmdb_movies'])

Showing Table: tmdb_movies


,index,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
index                26517 non-null int64
genre_ids            26517 non-null object
id                   26517 non-null int64
original_language    26517 non-null object
original_title       26517 non-null object
popularity           26517 non-null float64
release_date         26517 non-null object
title                26517 non-null object
vote_average         26517 non-null float64
vote_count           26517 non-null int64
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


None

----------------------------------------------------------------------------------------------------
finished


### This table appear well populated, but it will not be useful for our investigation.

### Connect `tmdb_movies`

In [ ]:
# connect to the table and view in pandas for cleaning
cur.execute('''SELECT *
                    FROM tmdb_movies
                    ;''')

clean_tmdb_movies_df = pd.DataFrame(cur.fetchall())
clean_tmdb_movies_df.columns = [x[0] for x in cur.description]
display(clean_tmdb_movies_df.dtypes)
clean_tmdb_movies_df.head(3)

### Convert `tmdb_movies`

In [ ]:
# convert `foreign_gross` to remove commas and to match `domestic_gross` data type
# this table seems fine, for now
# if needed, we can convert the date field in EDA


### Add `tmdb_movies` to sqlite

In [ ]:
# add the new df as a table to the sqlite database
clean_tmdb_movies_df.to_sql('clean_tmdb_tbl', conn, if_exists='replace')

# view names of all tables in the sql database to verify operation
conn.execute("select name from sqlite_master where type='table';").fetchall()

<h2 align='left'><font color='chocolate'>START QUESTIONS-------------here for reference-----remove when complete-----------------</font></h2>
<h2 align='center'><font color='chocolate'>Top Gross</font></h2>

<h3 align='center'><font color='coral'>by title</font></h3>

<h3 align='center'><font color='coral'>by studio</font></h3>

In [ ]:
# ... the 5 highest grossing studios
print("2018", bom_2018_df.groupby('studio').sum(
).sort_values(by=['Total_gross'], ascending=False).head())


print("-"*80)

print("2016-2018", bom_2016_18_df.groupby('studio').sum(
).sort_values(by=['Total_gross'], ascending=False).head())

#### Looks like we have an issue with `Year` values being summed.

***

<h2 align='center'><font color='chocolate'>DROP UNUSED AND UN-CLEANED TABLES FROM SQLITE DATABASE</font></h2>

<h3>Original data is still available in the csv files</h3>

* In use:
    `imdb_title_basics`
    `rt_movie_info`
    
* Cleaned:
    `clean_tn_tbl`
    `clean_bom_tbl`

<h3>Tables to remove</h3>    

* To drop:

     `bom_movie_gross`
     `imdb_name_basics`
     `imdb_title_akas`
     `imdb_title_basics`
     `imdb_title_crew`
     `imdb_title_principals`
     `imdb_title_ratings`
     `rt_movie_info`
     `rt_reviews`
     `tmdb_movies`
     `tn_movie_budgets`
     `clean_bom_tbl`
     `clean_tn_tbl`
     

In [ ]:
# Execute the DROP Table SQL statement

# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)

# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 
# dropTableStatement = "DROP TABLE demotable"
# cursor.execute(dropTableStatement)
 

# Close the connection object
# connection.close()

<h3 align='center'><font color='coral'>by title</font></h3>

<h3 align='center'><font color='coral'>by studio</font></h3>

<h3 align='center'><font color='coral'>by genre</font></h3>

***

***

<h2 align='center'><font color='chocolate'>Correlation</font></h2>

<h3>Question(s)</h3>

* First

<h3 align='center'><font color='coral'>gross, net</font></h3>

<h3 align='center'><font color='coral'>genre, gross</font></h3>

<h3 align='center'><font color='coral'>genre, net [only if gross, net not correlated</font></h3>

***

<h2 align='center'><font color='chocolate'>Top Studios</font></h2>

<h3>Question(s)</h3>

* First

<h3 align='center'><font color='coral'>gross genre mix</font></h3>

<h3 align='center'><font color='coral'>net genre mix</font></h3>

***

<h2 align='center'><font color='chocolate'>Final Recommendations</font></h2>

In [ ]:
# looking at ...
# plt.figure(figsize=(8, 5))
# sns.violinplot(x='year', y='Total_gross', data=bom_2016_18_df)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.style.use('ggplot')


data = [[2000, 2000, 2000, 2001, 2001, 2001, 2002, 2002, 2002],
        ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar'],
        [1, 2, 3, 4, 5, 6, 7, 8, 9]]

rows = zip(data[0], data[1], data[2])
headers = ['Year', 'Month', 'Value']
df = pd.DataFrame(rows, columns=headers)

df

In [ ]:
pivot_df = df.pivot(index='Year', columns='Month', values='Value')
pivot_df

In [ ]:
colors = ["#006D2C", "#31A354","#74C476"]
#Note: .loc[:,['Jan','Feb', 'Mar']] is used here to rearrange the layer ordering
pivot_df.loc[:,['Jan','Feb', 'Mar']].plot.barh(stacked=True, color=colors, figsize=(10,7))

In [ ]:
plt.bar()

In [ ]:
cur.close()
conn.close()